In [ ]:
from os import listdir, stat
from PIL import Image, ImageFile
import numpy as np
import tensorflow as tf
import math
from sklearn.model_selection import train_test_split
tf.test.is_gpu_available(
    cuda_only=True,
    min_cuda_compute_capability=None
)

In [ ]:
# Load pictures
path = "D:/Unity/Projects/screenshots"
directory = listdir(path)
X = np.empty(shape=(len(directory),108,192,3))
y = y=np.zeros(shape=(len(directory)))
for i,image in enumerate(directory):
    imagepath = path + "/" + image
    img = Image.open(imagepath)
    X[i]=np.asarray(img)
    # Read labels
    binary="000"
    if image[-5]=="1":
        binary="1"+binary[1:]
    if image[-6]=="1":
        binary=binary[0]+"1"+binary[2]
    if image[-7]=="1":
        binary=binary[:2]+"1"
    # Convert from multilabel to multiclass classification:
    y[i]=int(binary, 2)

In [ ]:
y = np.int_(y)
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=6)

In [ ]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 108, 192, 3])

    # Custom Convolution for Vertical Edge Detection (Sobel Filter)
    w = np.array([[-1,0,1],[-2,0,2],[-1,0,1]],dtype="float32")
    kernel = np.array([[w,w,w],[w,w,w],[w,w,w]])
    edge_v = tf.nn.conv2d(
        input=input_layer,
        filter=kernel,
        strides=[1,1,1,1],
        padding="SAME")
    #output shape [-1,108,192,3]
    
    # Custom Convolution for Horizontal Edge Detection (Sobel Filter)
    w = np.array([[1,2,1],[0,0,0],[-1,-2,-1]],dtype="float32")
    kernel = np.array([[w,w,w],[w,w,w],[w,w,w]])
    edge_h = tf.nn.conv2d(
        input=input_layer,
        filter=kernel,
        strides=[1,1,1,1],
        padding="SAME")
    #output shape [-1,108,192,3]
        
    sum1 = edge_v+edge_h+input_layer
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=10,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,108,192,32]

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[3,3], strides=[5,9])
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,22,22,10]

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=20,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,22,22,20]
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,11,11,20]

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 11 * 11 * 20])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4, 
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=8)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
# Set up Estimator
computer_vision = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./model2")

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
computer_vision.train(
    input_fn=train_input_fn,
    steps=11100)

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = computer_vision.evaluate(input_fn=eval_input_fn)
print(eval_results)